In [1]:
import pandas as pd
import numpy as np


In [2]:
# configuration setup

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"

people = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]
file_suffix = ".csv"
materials = ["PLA", "HYB"]

In [3]:
file_path_dict = [
    {
        "input": f"{BASE_DIR}/{material}/{person}25{material}.csv",
        "output": f"{OUTPUT_DIR}/{material}/{person}25{material}_kinetics.csv"
    }

    for person in people
    for material in materials
]

In [4]:
example_file = file_path_dict[0]["input"]
example_file

'~/Research/wheelchair/data/raw/Max/PLA/ALP25PLA.csv'

In [5]:
df = pd.read_csv(example_file)
df.head()

,time[sec],cycle[count],acc_x_R[m/s2],acc_y_R[m/s2],gyro_z_R[rad/s],force_x_R[N],force_y_R[N],force_z_R[N],moment_x_R[Nm],moment_y_R[Nm],...,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100
0,0.0000,1.0,9.301597,-2.38632,-0.087266,2.812102,4.992084,29.167699,-1.148323,-4.035011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0025,1.0,9.296733,-2.37168,-0.089394,3.026271,5.898925,29.833628,-1.143286,-4.119647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0050,1.0,9.247982,-2.28872,-0.095779,3.186506,6.378902,30.699335,-1.118095,-4.174404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0075,1.0,9.150399,-2.28872,-0.097908,3.614684,7.178919,30.899115,-1.123137,-4.229158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0100,1.0,9.204115,-2.28872,-0.102165,3.988977,7.818829,31.431858,-1.102987,-4.333697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df["time[sec]"][1]

np.float64(0.0025)

In [7]:
df.columns

Index(['time[sec]', 'cycle[count]', 'acc_x_R[m/s2]', 'acc_y_R[m/s2]',
       'gyro_z_R[rad/s]', 'force_x_R[N]', 'force_y_R[N]', 'force_z_R[N]',
       'moment_x_R[Nm]', 'moment_y_R[Nm]',
       ...
       'Unnamed: 91', 'Unnamed: 92', 'Unnamed: 93', 'Unnamed: 94',
       'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98',
       'Unnamed: 99', 'Unnamed: 100'],
      dtype='object', length=101)

In [8]:
# more helper functions
# note: these need to be run on the original dataframe, unfiltered

"""
Calculates the contact angle (deg)
  (Max theta_cop_[deg]) – (Min theta_cop_[deg])

  x: series data containing the angle
"""
def contact_angle(x):
    if len(x) == 0:
        return np.nan
    else:
        return x.dropna().max() - x.dropna().min()
    

"""
Calculates the contact ratio %
Contact time relative to the duration of one cycle
Params:
  x - the series data for the angle
"""
def calculate_contact_time(x):
    return 100*x.notna().mean()

#=============================== Calculation functions for the theta_cop =========================
def peak_torque_angle(side): # when the torque peaks
    idx = df[df['moment_z[W]'] == max(df['moment_z[W]'])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def peak_power_angle(side): # when the power peaks
    idx = df[df['power_z[W]'] == max(df['power_z[W]'])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def peak_tangential_force_angle(side): # when the tangential force is highest
    tangential_force_col = f"tangential_force_{side}[N]"
    idx = df[df[tangential_force_col] == max(df[tangential_force_col])]
    angle_col = f"theta_cop_{side}[deg]"
    return df.loc[idx, angle_col]

def angle_of_radial_force_change(df, side):
    # find when the radial force flips from positive to negative
    angle_col = f"theta_cop_{side}[deg]"
    radial_force_col = f"radial_force_{side}[N]"
    idx = first_pos_to_neg(df, radial_force_col)
    return df.loc[idx, angle_col]

def angle_of_axle_force_change(x):
    angle_col = f"theta_cop_{side}[deg]"
    axel_force_col = f"axle_force_{side}[N]"
    idx = first_pos_to_neg(df, axel_force_col)
    return df.loc[idx, angle_col]

def first_pos_to_neg(df, col):
    # Convert series to numpy array for speed
    vals = df[col].values
    time = df['time[sec]'].values
    
    # Loop through values to detect first positive -> negative
    for i in range(1, len(vals)):
        if vals[i-1] > 0 and vals[i] < 0:
            return time[i]
    
    # If no flip found
    return None


In [9]:
# Additional columns to add: power and total force
# add power the the calculations
df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']

# add the power[W] calculation for future calculations down the line
df['total_force_R[N]'] = np.sqrt(
    df["tangential_force_R[N]"]**2 +
    df["radial_force_R[N]"]**2 +
    df["axle_force_R[N]"]**2
)

df['total_force_L[N]'] =  np.sqrt(
    df["tangential_force_L[N]"]**2 +
    df["radial_force_L[N]"]**2 +
    df["axle_force_L[N]"]**2
)

In [14]:
kinematics_summary = df.groupby("cycle[count]").agg(**{
    "max_speed": ("speed_R[km/h]", "max"),
    "contact_angle_R": ("theta_cop_R[deg]", contact_angle),
    "contact_angle_L": ("theta_cop_L[deg]", contact_angle),
    "contact_time_R": ("theta_cop_R[deg]", calculate_contact_time),
    "contact_time_L": ("theta_cop_L[deg]", calculate_contact_time),
})
kinematics_summary

,max_speed,contact_angle_R,contact_angle_L,contact_time_R,contact_time_L
cycle[count],,,,,
1.0,9.867357,128.868829,126.312692,68.333333,64.722222
2.0,14.621420,182.662988,187.392446,49.808429,51.724138
3.0,18.310532,168.444841,325.874266,38.655462,45.798319
4.0,21.046794,176.040560,200.275463,34.821429,41.517857
5.0,23.390705,192.619522,351.738315,33.333333,37.333333
6.0,25.214879,200.324601,239.177742,27.777778,33.333333
7.0,27.008481,177.448709,198.332556,28.193833,28.634361
8.0,28.402089,180.764465,179.355869,27.947598,27.510917
9.0,29.614808,182.888508,171.736431,24.793388,24.380165


In [15]:
# Helper function to compute key metrics, for when the glove is still touching
def compute_cycle_metrics(df, side):
    """
    Compute key angles for each cycle where various metrics peak or change sign.
    """
    angle_col = f"theta_cop_{side}[deg]"
    torque_col = f"moment_z_{side}[Nm]"
    tangential_force_col = f"tangential_force_{side}[N]"
    radial_force_col = f"radial_force_{side}[N]"
    axle_force_col = f"axle_force_{side}[N]"

    # TODO: ask Yuki if this is a correct assumption --- how else to find the angle of the glove if it's not touching?
    # make sure that we only look at when we're touching the wheel
    df_copy = df[df[angle_col].notna()] 
    def peak_torque_angle(group):
        idx = group[torque_col].idxmax()
        return group.loc[idx, angle_col]
    
    def peak_power_angle(group):
        idx = group['power_z[W]'].idxmax()
        return group.loc[idx, angle_col]
    
    def peak_tangential_force_angle(group):
        idx = group[tangential_force_col].idxmax()
        return group.loc[idx, angle_col]
    
    def angle_of_radial_force_change(group):
        vals = group[radial_force_col].values
        angles = group[angle_col].values
        for i in range(1, len(vals)):
            if vals[i-1] > 0 and vals[i] < 0:
                return angles[i]

        return None
    
    def angle_of_axle_force_change(group):
        vals = group[axle_force_col].values
        angles = group[angle_col].values
        for i in range(1, len(vals)):
            if vals[i-1] > 0 and vals[i] < 0:
                return angles[i]
        return None
    
    results = df_copy.groupby('cycle[count]').apply(lambda g: pd.Series({
        f'peak_torque_angle_{side}': peak_torque_angle(g),
        f'peak_power_angle_{side}': peak_power_angle(g),
        f'peak_tangential_force_angle_{side}': peak_tangential_force_angle(g),
        f'radial_force_change_angle_{side}': angle_of_radial_force_change(g),
        f'axle_force_change_angle_{side}': angle_of_axle_force_change(g)
    }),
    include_groups=False)
    
    return results


In [16]:
compute_cycle_metrics(df, "R")

,peak_torque_angle_R,peak_power_angle_R,peak_tangential_force_angle_R,radial_force_change_angle_R,axle_force_change_angle_R
cycle[count],,,,,
1.0,89.306455,182.909307,101.314758,57.243319,NaN
2.0,89.901096,195.401238,107.564260,190.124486,NaN
3.0,79.260007,18.060560,98.531232,NaN,NaN
4.0,130.655705,23.162511,138.385804,NaN,NaN
5.0,121.758840,11.378775,129.732243,62.822353,NaN
6.0,136.795028,24.182415,138.679283,9.632199,NaN
7.0,148.736809,15.098501,144.299824,NaN,NaN
8.0,62.996271,14.517081,140.742295,116.202342,NaN
9.0,154.909763,19.638195,154.909763,NaN,NaN
